In [1]:
!pip install datasets
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

<ipython-input-2-3409770de00f>:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:

from huggingface_hub import HfApi


huggingface_user = "shrey209"
dataset_repo = f"{huggingface_user}/data-jsonl"

api = HfApi()
api.create_repo(repo_id=dataset_repo, repo_type="dataset", exist_ok=True)


api.upload_file(
    path_or_fileobj="/content/data.jsonl",
    path_in_repo="data.jsonl",
    repo_id=f"{huggingface_user}/data-jsonl",
    repo_type="dataset"
)



data.jsonl:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shrey209/data-jsonl/commit/31be2a48b40b2b61bed8bb81ca27d2c23de9f7e6', commit_message='Upload data.jsonl with huggingface_hub', commit_description='', oid='31be2a48b40b2b61bed8bb81ca27d2c23de9f7e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/shrey209/data-jsonl', endpoint='https://huggingface.co', repo_type='dataset', repo_id='shrey209/data-jsonl'), pr_revision=None, pr_num=None)

In [4]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

huggingface_user = "shrey209"

config = {
    "model_config": {
        "base_model": "unsloth/llama-3-8b-Instruct-bnb-4bit",
        "max_seq_length": 1024,
        "dtype": torch.float16,
        "load_in_4bit": True,
    },
    "lora_config": {
        "r": 16,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                           "gate_proj", "up_proj", "down_proj"],
        "lora_alpha": 16,
        "lora_dropout": 0,
        "bias": "none",
        "use_gradient_checkpointing": True,
        "use_rslora": False,
        "use_dora": False,
        "loftq_config": None,
    },
}


print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config["model_config"]["base_model"],
    max_seq_length=config["model_config"]["max_seq_length"],
    dtype=config["model_config"]["dtype"],
    load_in_4bit=config["model_config"]["load_in_4bit"],
)


model = FastLanguageModel.get_peft_model(
    model,
    r=config["lora_config"]["r"],
    target_modules=config["lora_config"]["target_modules"],
    lora_alpha=config["lora_config"]["lora_alpha"],
    lora_dropout=config["lora_config"]["lora_dropout"],
    bias=config["lora_config"]["bias"],
    use_gradient_checkpointing=config["lora_config"]["use_gradient_checkpointing"],
    random_state=42,
    use_rslora=config["lora_config"]["use_rslora"],
    use_dora=config["lora_config"]["use_dora"],
    loftq_config=config["lora_config"]["loftq_config"],
)


model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

print("Model and tokenizer saved successfully.")

Loading model...
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model and tokenizer saved successfully.


In [6]:
import os
import torch
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

os.environ["WANDB_DISABLED"] = "true"  # ✅ Disable Weights & Biases logging

# ✅ Load dataset
dataset_name = "shrey209/data-jsonl"
dataset_train = load_dataset(dataset_name, split="train").select(range(5))

# ✅ Fix: Assign a padding token
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# ✅ Preprocessing Function (move it **above** the first `.map()` call)
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,  # ✅ Use 1024 for LLaMA 3
    )
    model_inputs["labels"] = model_inputs["input_ids"][:]  # Copy input IDs as labels
    return model_inputs

# ✅ Tokenize Dataset (after setting pad_token)
tokenized_datasets = dataset_train.map(preprocess_function, batched=True, remove_columns=["text"])

# ✅ Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    warmup_steps=0,
    num_train_epochs=0.2,
    learning_rate=2e-4,
    fp16=True,  # ✅ Set to True since we're using FP16
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    output_dir="outputs",
    save_strategy="no",
    push_to_hub=False,
    report_to="none",
)

# ✅ Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# ✅ Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Start Training
trainer.train()


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

<ipython-input-6-4f4d41918220>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,7.760300


TrainOutput(global_step=1, training_loss=7.760346412658691, metrics={'train_runtime': 24.3985, 'train_samples_per_second': 0.041, 'train_steps_per_second': 0.041, 'total_flos': 92735910445056.0, 'train_loss': 7.760346412658691, 'epoch': 0.3333333333333333})

In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = config.get("model_config").get("finetuned_model"),
        max_seq_length = config.get("model_config").get("max_seq_length"),
        dtype = config.get("model_config").get("dtype"),
        load_in_4bit = config.get("model_config").get("load_in_4bit"),
    )


FastLanguageModel.for_inference(model)

system_prompt = f"you are a cyber security expert explain :-Compare and contrast the security features of Apache, IIS, and nginx web servers"


prompt = input('TYPE PROMPT TO LLAMA3: ')
inputs = tokenizer(
[
    f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|end_header_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)